In [1]:
import numpy as np
import pandas as pd
import mnist

data_folder  = '../../data/mnist/'

In [4]:
# from glob import glob
# import os

# data_train = pd.read_csv(data_folder + 'train.csv')

mnist.init()
X_train, y_train, X_test, y_test = mnist.load()

Download complete.
Save complete.


In [150]:
X_train.min(1)

array([0, 0, 0, ..., 0, 0, 0], dtype=uint8)

In [6]:
import pickle

with open('X_train_mnist.pkl', 'wb') as handle:
    pickle.dump(X_train, handle)
with open('X_test_mnist.pkl', 'wb') as handle:
    pickle.dump(X_test, handle)
with open('y_train_mnist.pkl', 'wb') as handle:
    pickle.dump(y_train, handle)
with open('y_test_mnist.pkl', 'wb') as handle:
    pickle.dump(y_test, handle)

In [ ]:
import pickle

with open('X_train_mnist.pkl', 'rb') as handle:
    X_train = pickle.load(handle)
with open('X_test_mnist.pkl', 'rb') as handle:
    X_test = pickle.load(handle)
with open('y_train_mnist.pkl', 'rb') as handle:
    y_train = pickle.load(handle)
with open('y_test_mnist.pkl', 'rb') as handle:
    y_test = pickle.load(handle)

In [142]:
y_train[:10]

array([5, 0, 4, 1, 9, 2, 1, 3, 1, 4], dtype=uint8)

In [5]:
features, label = data_train.drop(['label'], axis=1).values, data_train['label'].values.reshape(-1, 1)

In [6]:
features.shape, label.shape

((42000, 784), (42000, 1))

In [250]:
from collections import namedtuple
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F
from collections import defaultdict

class AutoencoderKmeans(nn.Module):
    def __init__(self, input_dim, encoding_dim=20, num_clusters=10):
        super().__init__()
        # Global timestamp
        self.t = 0
        
        # Hyper parameters
        self.num_clusters = num_clusters
        self.encoding_dim = encoding_dim
        
        # Define AE architecture
        self.fc1 = nn.Linear(input_dim, self.encoding_dim)
        self.fc2 = nn.Linear(self.encoding_dim, input_dim)
        
        # Initialize cluster centers
        self.centers = torch.from_numpy(np.random.uniform(0, 1, (self.num_clusters, self.encoding_dim))).type(torch.FloatTensor)
        self.clusters = None
        self.AKoutput = namedtuple('AKoutput', 'k_means_loss clusters data_hidden output')
        
    def forward(self, x):
        x = self.fc1(x)
        
        data_hidden = x.clone()
        clusters = self.get_clusters(data_hidden)
        k_means_loss = self.get_kmeans_loss(data_hidden, clusters)
        self.centers = self.update_centers(data_hidden, clusters)
        
        x = self.fc2(x)

        output = x
        
        self.t += 1
        
        output_tuple = self.AKoutput(k_means_loss=k_means_loss, clusters=clusters, data_hidden=data_hidden, output=output)
#         return k_means_loss, clusters, data_hidden, output
        return output_tuple
    
    def get_clusters(self, dataset):
        clusters = []
        if isinstance(self.centers, torch.autograd.Variable):
            self.centers = self.centers.data
        for data in dataset[:10000]:
            clusters.append(((data - Variable(self.centers, requires_grad=False))**2).sum(1).min(0)[1][0])
        return clusters
    
    def get_kmeans_loss(self, dataset, clusters):
        loss = [0 for i in range(self.num_clusters)]
        
        for i, cluster in enumerate(clusters):
            loss[int(cluster.data)] += ((dataset[i] - Variable(self.centers[int(cluster.data)], requires_grad=False))**2).sum()
            
        return sum(loss)
    
    def update_centers(self, dataset, clusters):
        data_by_cluster = [[] for i in range(self.num_clusters)]
        
        for i, x in enumerate(clusters):
            data_by_cluster[int(x.data)].append(dataset[i])
        
#         print(len(data_by_cluster))
        
#         for i in range(len(data_by_cluster)):
#             print(len(data_by_cluster[i]))
        
        centers = [sum(data) / len(data) for data in data_by_cluster]
        centers = torch.stack(centers)
        
        return centers
        
    @staticmethod
    def __square_loss(v1, v2):
        loss = ((v1 - v2)**2).sum(1)[0]
        
        return loss        

In [251]:
input_dim = 784

model = AutoencoderKmeans(input_dim)

In [252]:
print(len(list(model.parameters())))
print(list(model.parameters())[0].size())
print(list(model.parameters())[1].size())
print(list(model.parameters())[2].size())
print(list(model.parameters())[3].size())

4
torch.Size([20, 784])
torch.Size([20])
torch.Size([784, 20])
torch.Size([784])


In [253]:
criterion = nn.MSELoss()
learning_rate = 0.001
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
lambd = 0.1

In [254]:
num_epochs = 1000

In [255]:
recon_losses = []
kmeans_losses = []

for epoch in range(num_epochs):
    x_torch = torch.from_numpy(X_train).type(torch.FloatTensor)
    y_torch = torch.from_numpy(X_train).type(torch.FloatTensor)

    x = Variable(x_torch)
    y = Variable(y_torch)

    optimizer.zero_grad()

#     k_means_loss, _, outputs = model(x)
    output_tuple = model(x)
    cost = criterion(output_tuple.output, y) + 0.00001 * output_tuple.k_means_loss
#     cost = criterion(outputs, y) + 0.00001 * k_means_loss
    cost.backward()

    optimizer.step()
    
    recon_loss = float(((y_torch - output_tuple.output.data)**2).mean())
    kmeans_loss = float(0.00001 * output_tuple.k_means_loss)
    print("Epoch {}".format(epoch))
    print("Reconstruction Loss: {0}, K-means Loss: {1}".format(recon_loss, kmeans_loss))
    recon_losses.append(recon_loss)
    kmeans_losses.append(kmeans_loss)
#     print(((y_torch - outputs.data)**2).mean())
#     print(0.00001 * k_means_loss)

Epoch 0
Reconstruction Loss: 8365.362976209253, K-means Loss: 5452.10546875
Epoch 1
Reconstruction Loss: 7904.123085062411, K-means Loss: 2796.222900390625
Epoch 2
Reconstruction Loss: 7714.672661175864, K-means Loss: 1983.618408203125
Epoch 3
Reconstruction Loss: 7618.877060629487, K-means Loss: 1657.6378173828125
Epoch 4
Reconstruction Loss: 7551.840899174778, K-means Loss: 1463.075927734375
Epoch 5
Reconstruction Loss: 7499.758177631087, K-means Loss: 1324.2828369140625
Epoch 6
Reconstruction Loss: 7456.640709946238, K-means Loss: 1216.552001953125
Epoch 7
Reconstruction Loss: 7419.561067788388, K-means Loss: 1130.08154296875
Epoch 8
Reconstruction Loss: 7386.525427188499, K-means Loss: 1058.3455810546875
Epoch 9
Reconstruction Loss: 7356.04713053859, K-means Loss: 997.6681518554688
Epoch 10
Reconstruction Loss: 7326.881732993461, K-means Loss: 945.389892578125
Epoch 11
Reconstruction Loss: 7298.084325823398, K-means Loss: 899.427978515625
Epoch 12
Reconstruction Loss: 7269.04481653

Epoch 102
Reconstruction Loss: 4131.440363491141, K-means Loss: 164.91831970214844
Epoch 103
Reconstruction Loss: 4129.847716716681, K-means Loss: 162.7261199951172
Epoch 104
Reconstruction Loss: 4128.366109304334, K-means Loss: 160.59738159179688
Epoch 105
Reconstruction Loss: 4126.939202342303, K-means Loss: 158.61216735839844
Epoch 106
Reconstruction Loss: 4125.585361741569, K-means Loss: 156.7139434814453
Epoch 107
Reconstruction Loss: 4124.137675982547, K-means Loss: 155.1031951904297
Epoch 108
Reconstruction Loss: 4122.609544792106, K-means Loss: 153.6694793701172
Epoch 109
Reconstruction Loss: 4121.070086120972, K-means Loss: 152.3441619873047
Epoch 110
Reconstruction Loss: 4119.571693845858, K-means Loss: 151.1218719482422
Epoch 111
Reconstruction Loss: 4118.181804165205, K-means Loss: 149.90374755859375
Epoch 112
Reconstruction Loss: 4116.92357943993, K-means Loss: 148.6549530029297
Epoch 113
Reconstruction Loss: 4115.766551298723, K-means Loss: 147.40878295898438
Epoch 114
Re

Epoch 201
Reconstruction Loss: 4057.0919883548154, K-means Loss: 101.53502655029297
Epoch 202
Reconstruction Loss: 4056.1977790729547, K-means Loss: 101.52021026611328
Epoch 203
Reconstruction Loss: 4055.2949368615164, K-means Loss: 101.50951385498047
Epoch 204
Reconstruction Loss: 4054.3724900692628, K-means Loss: 101.5184097290039
Epoch 205
Reconstruction Loss: 4053.442099709534, K-means Loss: 101.53080749511719
Epoch 206
Reconstruction Loss: 4052.4911127529053, K-means Loss: 101.56431579589844
Epoch 207
Reconstruction Loss: 4051.5364965611443, K-means Loss: 101.59666442871094
Epoch 208
Reconstruction Loss: 4050.5605223088914, K-means Loss: 101.65213775634766
Epoch 209
Reconstruction Loss: 4049.5815840476607, K-means Loss: 101.70524597167969
Epoch 210
Reconstruction Loss: 4048.579292014539, K-means Loss: 101.78559112548828
Epoch 211
Reconstruction Loss: 4047.5785084197546, K-means Loss: 101.86019897460938
Epoch 212
Reconstruction Loss: 4046.547758883615, K-means Loss: 101.97209930419

Epoch 300
Reconstruction Loss: 3901.089508176899, K-means Loss: 140.88389587402344
Epoch 301
Reconstruction Loss: 3898.416142942807, K-means Loss: 142.16551208496094
Epoch 302
Reconstruction Loss: 3895.587309499308, K-means Loss: 143.53611755371094
Epoch 303
Reconstruction Loss: 3892.425327001123, K-means Loss: 145.09556579589844
Epoch 304
Reconstruction Loss: 3888.934872213175, K-means Loss: 146.73123168945312
Epoch 305
Reconstruction Loss: 3884.8515040683365, K-means Loss: 148.55445861816406
Epoch 306
Reconstruction Loss: 3880.1023441677844, K-means Loss: 150.6791534423828
Epoch 307
Reconstruction Loss: 3874.4788291961145, K-means Loss: 153.4066619873047
Epoch 308
Reconstruction Loss: 3868.1601143183807, K-means Loss: 156.7845916748047
Epoch 309
Reconstruction Loss: 3861.3552484639686, K-means Loss: 160.643798828125
Epoch 310
Reconstruction Loss: 3854.4731234045935, K-means Loss: 164.4237823486328
Epoch 311
Reconstruction Loss: 3847.754673199168, K-means Loss: 167.46505737304688
Epoc

Epoch 399
Reconstruction Loss: 3595.6480581808582, K-means Loss: 218.51129150390625
Epoch 400
Reconstruction Loss: 3593.285185481646, K-means Loss: 219.3797607421875
Epoch 401
Reconstruction Loss: 3591.721434718413, K-means Loss: 219.4983367919922
Epoch 402
Reconstruction Loss: 3589.346924072783, K-means Loss: 220.4402313232422
Epoch 403
Reconstruction Loss: 3587.8447238084746, K-means Loss: 220.5693817138672
Epoch 404
Reconstruction Loss: 3585.4353464066894, K-means Loss: 221.6192169189453
Epoch 405
Reconstruction Loss: 3583.9699085232573, K-means Loss: 221.78111267089844
Epoch 406
Reconstruction Loss: 3581.5710808880417, K-means Loss: 222.88790893554688
Epoch 407
Reconstruction Loss: 3580.1458001857154, K-means Loss: 223.05117797851562
Epoch 408
Reconstruction Loss: 3577.7845815256474, K-means Loss: 224.13539123535156
Epoch 409
Reconstruction Loss: 3576.3820579839053, K-means Loss: 224.2340087890625
Epoch 410
Reconstruction Loss: 3574.007700733709, K-means Loss: 225.2611541748047
Epo

...

Epoch 597
Reconstruction Loss: 3192.228329310033, K-means Loss: 298.35186767578125
Epoch 598
Reconstruction Loss: 3189.800125665618, K-means Loss: 299.1777038574219
Epoch 599
Reconstruction Loss: 3188.716235371192, K-means Loss: 298.6969299316406
Epoch 600
Reconstruction Loss: 3186.1946293499323, K-means Loss: 299.5812072753906
Epoch 601
Reconstruction Loss: 3185.153441303385, K-means Loss: 298.95745849609375
Epoch 602
Reconstruction Loss: 3182.5891583485045, K-means Loss: 299.93890380859375
Epoch 603
Reconstruction Loss: 3181.7949342762395, K-means Loss: 299.23358154296875
Epoch 604
Reconstruction Loss: 3179.2997132949654, K-means Loss: 300.2451477050781
Epoch 605
Reconstruction Loss: 3178.7452929522924, K-means Loss: 299.4448547363281
Epoch 606
Reconstruction Loss: 3175.879267404725, K-means Loss: 300.9691162109375
Epoch 607
Reconstruction Loss: 3175.401918998702, K-means Loss: 300.1918029785156
Epoch 608
Reconstruction Loss: 3172.379279868936, K-means Loss: 301.95367431640625
Epoch 

Epoch 697
Reconstruction Loss: 3016.160455011771, K-means Loss: 322.4096374511719
Epoch 698
Reconstruction Loss: 3011.9679999933082, K-means Loss: 325.103759765625
Epoch 699
Reconstruction Loss: 3012.5251826915246, K-means Loss: 323.14019775390625
Epoch 700
Reconstruction Loss: 3008.0592790008895, K-means Loss: 326.0894470214844
Epoch 701
Reconstruction Loss: 3008.7088599100925, K-means Loss: 323.96771240234375
Epoch 702
Reconstruction Loss: 3004.331206952564, K-means Loss: 326.66741943359375
Epoch 703
Reconstruction Loss: 3004.82548740011, K-means Loss: 324.6186828613281
Epoch 704
Reconstruction Loss: 3000.6226037548136, K-means Loss: 327.3043518066406
Epoch 705
Reconstruction Loss: 3001.327613131182, K-means Loss: 325.13189697265625
Epoch 706
Reconstruction Loss: 2997.368324745397, K-means Loss: 327.644287109375
Epoch 707
Reconstruction Loss: 2998.295379854503, K-means Loss: 325.35394287109375
Epoch 708
Reconstruction Loss: 2994.359201113939, K-means Loss: 327.89263916015625
Epoch 70

Epoch 797
Reconstruction Loss: 2885.5013146406914, K-means Loss: 328.99542236328125
Epoch 798
Reconstruction Loss: 2881.3929652769048, K-means Loss: 332.03314208984375
Epoch 799
Reconstruction Loss: 2883.339468857054, K-means Loss: 329.1091003417969
Epoch 800
Reconstruction Loss: 2879.2220626202734, K-means Loss: 332.13995361328125
Epoch 801
Reconstruction Loss: 2881.16991618441, K-means Loss: 329.1974182128906
Epoch 802
Reconstruction Loss: 2877.0123244019874, K-means Loss: 332.2646179199219
Epoch 803
Reconstruction Loss: 2878.957519351007, K-means Loss: 329.26019287109375
Epoch 804
Reconstruction Loss: 2874.8128454954212, K-means Loss: 332.24658203125
Epoch 805
Reconstruction Loss: 2876.7556493694187, K-means Loss: 329.2284240722656
Epoch 806
Reconstruction Loss: 2872.664054062905, K-means Loss: 332.1753845214844
Epoch 807
Reconstruction Loss: 2874.553553091842, K-means Loss: 329.1670837402344
Epoch 808
Reconstruction Loss: 2870.504577024742, K-means Loss: 331.9706115722656
Epoch 809

Epoch 897
Reconstruction Loss: 2781.9893117828665, K-means Loss: 327.2845764160156
Epoch 898
Reconstruction Loss: 2779.9363392308733, K-means Loss: 328.580810546875
Epoch 899
Reconstruction Loss: 2780.355424082461, K-means Loss: 327.4367370605469
Epoch 900
Reconstruction Loss: 2778.098683087132, K-means Loss: 328.99090576171875
Epoch 901
Reconstruction Loss: 2778.768456286996, K-means Loss: 327.649658203125
Epoch 902
Reconstruction Loss: 2776.245433315824, K-means Loss: 329.5419616699219
Epoch 903
Reconstruction Loss: 2777.273940449748, K-means Loss: 327.95843505859375
Epoch 904
Reconstruction Loss: 2774.4835401088467, K-means Loss: 330.2423095703125
Epoch 905
Reconstruction Loss: 2775.956071605917, K-means Loss: 328.39251708984375
Epoch 906
Reconstruction Loss: 2772.8095087299407, K-means Loss: 331.1636657714844
Epoch 907
Reconstruction Loss: 2774.736227694822, K-means Loss: 328.9455261230469
Epoch 908
Reconstruction Loss: 2771.175513807329, K-means Loss: 332.2886047363281
Epoch 909
R

Epoch 997
Reconstruction Loss: 2694.8921913752174, K-means Loss: 339.8724365234375
Epoch 998
Reconstruction Loss: 2698.0573215977847, K-means Loss: 333.9107971191406
Epoch 999
Reconstruction Loss: 2692.514554509056, K-means Loss: 336.900146484375


In [257]:
# X_train_hidden, _ = model(Variable(torch.from_numpy(X_train.values).type(torch.FloatTensor)))
# X_test_hidden, _ = model(Variable(torch.from_numpy(X_test.values).type(torch.FloatTensor)))

output_tuple_train = model(Variable(torch.from_numpy(X_train).type(torch.FloatTensor)))
output_tuple_test = model(Variable(torch.from_numpy(X_test).type(torch.FloatTensor)))

In [259]:
output_tuple_train.output[:10]

Variable containing:
-4.9652e-01  8.7988e-01 -3.6134e-01  ...   1.3048e+00  1.1890e+00  1.0158e+00
 1.0322e+00  2.0356e+00  4.6983e-01  ...   1.4306e+00  2.0928e+00  1.1317e+00
-9.6654e-02  1.7831e+00  3.6032e-01  ...   1.1337e+00 -3.4721e-01  7.4321e-01
                ...                   ⋱                   ...                
-1.8232e+00 -1.0643e+00  1.4060e+00  ...  -4.3853e-01 -4.8646e-01  3.7386e-01
-3.6603e-01 -5.4236e-01  1.5061e+00  ...   4.1809e-01 -4.3590e-01  1.2563e+00
-2.3730e+00  9.7460e-01 -1.1650e+00  ...  -2.5269e+00 -1.5336e+00 -1.0853e+00
[torch.FloatTensor of size 10x784]

In [258]:
output_tuple_train.clusters[:10]

[Variable containing:
  8
 [torch.LongTensor of size 1], Variable containing:
  3
 [torch.LongTensor of size 1], Variable containing:
  9
 [torch.LongTensor of size 1], Variable containing:
  4
 [torch.LongTensor of size 1], Variable containing:
  6
 [torch.LongTensor of size 1], Variable containing:
  7
 [torch.LongTensor of size 1], Variable containing:
  4
 [torch.LongTensor of size 1], Variable containing:
  1
 [torch.LongTensor of size 1], Variable containing:
  4
 [torch.LongTensor of size 1], Variable containing:
  6
 [torch.LongTensor of size 1]]

In [88]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()

lr.fit(X_train, y_train)
y_pred = lr.predict_proba(X_test)

/home/calvinku/anaconda3/envs/python3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [89]:
from sklearn.metrics import roc_auc_score

print(roc_auc_score(y_test, y_pred[:,1]))

0.8317708333333333


In [90]:
X_train_hidden = X_train_hidden.data.numpy()
X_test_hidden = X_test_hidden.data.numpy()


lr = LogisticRegression()

lr.fit(X_train_hidden, y_train)
y_pred = lr.predict_proba(X_test_hidden)

/home/calvinku/anaconda3/envs/python3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [91]:
print(roc_auc_score(y_test, y_pred[:,1]))

0.8052083333333334


In [20]:
x = Variable(torch.ones(1), requires_grad=True)
y = 5 * (x + 1) ** 2

In [21]:
y

Variable containing:
 20
[torch.FloatTensor of size 1]

In [22]:
o = 0.5 * torch.sum(y)

In [23]:
o

Variable containing:
 10
[torch.FloatTensor of size 1]

In [24]:
o.backward()

In [25]:
x.grad

Variable containing:
 10
[torch.FloatTensor of size 1]

In [35]:
np.random.rand(100).mean()

0.48660327761141575

In [46]:
centers = torch.from_numpy(np.random.rand(10, 784)).type(torch.FloatTensor)

In [47]:
dataset = torch.from_numpy(np.random.rand(20000, 784)).type(torch.FloatTensor)

In [48]:
dataset.shape

torch.Size([20000, 784])

In [81]:
((dataset[0] - centers)**2).sum(1).mean()

126.69114151000977

In [69]:
((dataset[0] - centers)**2).sum(1).max(0)

(
  135.5187
 [torch.FloatTensor of size 1], 
  2
 [torch.LongTensor of size 1])

In [86]:
[((data - centers)**2).sum(1).min(0)[1][0] for data in dataset][:10]

[4, 0, 3, 3, 6, 4, 4, 0, 4, 9]

In [84]:
([((data - centers)**2).sum(1) for data in dataset][:10])

[
  119.5014
  123.2441
  135.5187
  129.4469
  117.6147
  128.2050
  122.8010
  129.3588
  126.3594
  134.8614
 [torch.FloatTensor of size 10], 
  125.4285
  130.5832
  135.6001
  138.1736
  135.7134
  129.4753
  133.7235
  141.2342
  144.9864
  134.1505
 [torch.FloatTensor of size 10], 
  133.7800
  128.7815
  131.9692
  122.2993
  129.3626
  133.6053
  130.1130
  132.8036
  134.6575
  125.4915
 [torch.FloatTensor of size 10], 
  131.9848
  131.0141
  134.8151
  125.2208
  128.1747
  132.2542
  136.5810
  137.9877
  132.6785
  127.1587
 [torch.FloatTensor of size 10], 
  137.0968
  132.0614
  129.9968
  133.9629
  135.1674
  135.2247
  125.4944
  128.4681
  134.7811
  143.6792
 [torch.FloatTensor of size 10], 
  127.8875
  122.6231
  127.5521
  125.7077
  118.4951
  125.3392
  132.6665
  123.1378
  127.9611
  131.0582
 [torch.FloatTensor of size 10], 
  135.2402
  136.2485
  135.2043
  130.9381
  125.1189
  134.3459
  155.2023
  127.7248
  138.9448
  131.5292
 [torch.FloatTensor of s

In [92]:
v1 = torch.from_numpy(np.random.rand(1, 15)).type(torch.FloatTensor)
v2 = torch.from_numpy(np.random.rand(1, 15)).type(torch.FloatTensor)

def square_loss(v1, v2):
    loss = ((v1 - v2)**2).sum(1)[0]
    
    return loss

square_loss(v1, v2)

2.0329911708831787